''''sh
 pip install requests ipykernel Pillow     
''''

In [ ]:
# CONFIG

__longitude__ = 9.37477
__latitude__ = 47.42391

import risks
import img_generation

In [ ]:
# Example parameters from an API or other source
forecast_type=risks.forecast_type_dict["Daily"]
longitude=__longitude__
latitude=__latitude__
measureLabel=[
        "TempAir_DailyMax (C)",
        "TempAir_DailyMin (C)",
        "TempAir_DailyAvg (C)",
        "Precip_DailySum (mm)",
        "Evapotranspiration_DailySum (mm)",
        "Soilmoisture_0to10cm_DailyAvg (vol%)",
]
start_date="2025-03-20"
end_date="2025-03-20"

weather_forecast = risks.get_weather_forecast(
    forecast_type=forecast_type,
    longitude=longitude, 
    latitude=latitude,
    measureLabel=measureLabel,
    start_date=start_date,
    end_date=end_date
)

crop_name = 'Corn'
TMAX = weather_forecast["TempAir_DailyMax (C)"]  # Maximum temperature
TMIN = weather_forecast["TempAir_DailyMin (C)"]  # Minimum temperature
TAVG = weather_forecast["TempAir_DailyAvg (C)"]  # Average temperature
P = weather_forecast["Precip_DailySum (mm)"]     # Cumulative rainfall
SM = weather_forecast["Soilmoisture_0to10cm_DailyAvg (vol%)"]   # Soil moisture
# GDD calculation is internal to the calculator
E = 300    # Cumulative evaporation
pH = 6.5   # Soil pH
N = 0.02   # Available nitrogen

# Create an instance of the calculator with the parameters
calculator = risks.CropRisksCalculator(crop_name, TMAX, TMIN, TAVG, P, E, SM, pH, N)

# Calculate stresses and risks
risks, results = calculator.calculate(binarize=True)

# Print results
print(results)
print(risks)

{'diurnal_stress': 0, 'nighttime_stress': 0, 'frost_stress': 0, 'drought_risk': 0, 'yield_risk': 3073493.6184212}
{'diurnal_stress': 0, 'nighttime_stress': 0, 'frost_stress': 0, 'drought_risk': 0, 'yield_risk': 1}


In [ ]:

elements=[
    {"header": "Elevated temperature", "body": "The maximum temperature will be exceeded, I recommend applyig Stress Buster to ensure the well being of your crops.", "type": "warning", "icon": "Icons/temperature-arrow-up-solid.png", "status":(risks['diurnal_stress']|risks["nighttime_stress"])},
    {"header": "Low moisture", "body": "A drought is incoming, I recomend appling Stress buster to ensure the well being of your crops.", "type": "warning", "icon": "Icons/sun-plant-wilt-solid.png","status":risks["frost_stress"]},
    {"header": "Frost Warning", "body": "Frost is expected in the coming days, I recommend applying Stress Buster to your crops to ensure their well being", "type": "warning", "icon": "Icons/snowflake-regular.png","status":risks["drought_risk"]},
    {"header": "Yield Risk", "body":"Based on avilible data, your yield is at a risk of being lower that expected. Apply Yield Booster to your crops to ensure the best possible yield.", "type": "warning", "icon": "Icons/arrow-down-wide-short-solid.png","status":risks["yield_risk"]}
    ]
city_name = risks.get_city_name(__longitude__, __latitude__)

img = img_generation.generate(elements, crop_name, city_name)
img.show()

Icons/temperature-arrow-up-solid.png
Icons/sun-plant-wilt-solid.png
Icons/snowflake-regular.png
Icons/arrow-down-wide-short-solid.png
